In [2]:
from transformers import AutoTokenizer, AutoModel

model_llm = "Writer/camel-5b-hf"
model_emb = "BAAI/bge-small-en-v1.5"

save_llm = "../models/camel-5b-hf"
save_emb = "../models/bge-small-en-v1.5"

tokenizer = AutoTokenizer.from_pretrained(model_llm)
model = AutoModel.from_pretrained(model_llm)
tokenizer.save_pretrained(save_llm)
model.save_pretrained(save_llm)

tokenizer = AutoTokenizer.from_pretrained(model_emb)
model = AutoModel.from_pretrained(model_emb)
tokenizer.save_pretrained(save_emb)
model.save_pretrained(save_emb)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [4]:
from llama_index.llms.huggingface import HuggingFaceLLM


llm = HuggingFaceLLM(
    tokenizer_name = save_llm,
    model_name = save_llm,
    context_window = 2048,
    max_new_tokens = 512,
    generate_kwargs={"temperature": 0.25, "do_sample" : True},
    device_map = "auto",
    tokenizer_kwargs={"max_length" : 2048}
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
llm.complete("What is GPT4")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CompletionResponse(text='?\nGPT4 is a machine learning algorithm developed by OpenAI, which generates human-like text responses from a set of pre-defined rules. It is based on the GPT model, which is a natural language processing (NLP) model developed by OpenAI. GPT4 is widely used in various applications, such as text summarization, chatbots, and virtual assistants.', additional_kwargs={}, raw={'model_output': tensor([[ 2061,   318,   402, 11571,    19,    30,   198,    38, 11571,    19,
           318,   257,  4572,  4673, 11862,  4166,   416,  4946, 20185,    11,
           543, 18616,  1692,    12,  2339,  2420,  9109,   422,   257,   900,
           286,   662,    12, 23211,  3173,    13,   632,   318,  1912,   319,
           262,   402, 11571,  2746,    11,   543,   318,   257,  3288,  3303,
          7587,   357,    45, 19930,     8,  2746,  4166,   416,  4946, 20185,
            13,   402, 11571,    19,   318,  6768,   973,   287,  2972,  5479,
            11,   884,   355,  2

In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name=save_emb)

In [8]:
embeddings = embed_model.get_text_embedding("What is GPT4")
embeddings[:3]

[0.0809716209769249, 0.046074219048023224, 0.024222958832979202]

In [13]:
!ls dataset/

llamaindex_data


## Llamaindex with local models

In [15]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("dataset/llamaindex_data").load_data()
vector_index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, similarity_top_k = 1)

In [17]:
query_engine = vector_index.as_query_engine(llm = llm)

In [18]:
# How to convert it to local machine

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("dataset/llamaindex_data").load_data()
vector_index = VectorStoreIndex.from_documents(documents, embed_model=f"local:{save_emb}", similarity_top_k = 1)

In [19]:
response = query_engine.query("What is Mistral AI?")
response.response

Token indices sequence length is longer than the specified maximum sequence length for this model (1514 > 512). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' Mistral AI is a French AI company founded in 2015 by Yann LeCun, Béatrice Prade, and Vincent Dumoulin. It is known for its work on facial recognition technology, particularly in the field of autonomous vehicles. Mistral AI is headquartered in Paris, France.'

### Get Documents

In [21]:
import requests

def get_wikipedia_page_links(title):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"
  params = {
      "action" : "query",
      "format" : "json",
      "titles" : title,
      "prop" : "links",
      "pllimit": "max"
  }

  all_links = []

  while True:
    response = S.get(url = URL, params = params).json()
    pages = response.get("query", {}).get("pages", {})

    for page_id, page_content in pages.items():
      links = page_content.get("links", [])

      for link in links:
        all_links.append(link.get("title"))
        links = page_content.get("links", [])

        for link in links:
          all_links.append(link.get("title"))

      if "continue" in response:
        params["plcontinue"] = response["continue"]["plcontinue"]
      else:
        break

    return all_links


In [22]:
links_lm = get_wikipedia_page_links("Large language model")
len(links_lm)

250500

In [31]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key = api_key)

def get_response(prompt):
  response = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages = [
          {"role" : "system", "content" : prompt }, ]
  )
  return response.choices[0].message.content


In [33]:
def get_article_abstract(title):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"
  params = {
      "action" : "query",
      "format" : "json",
      "titles" : title,
      "prop" : "extracts",
      "exintro" : True,
      "explaintext": True
  }
  response = requests.get(URL, params).json()
  page = next(iter(response["query"]["pages"].values()))

  return page.get("extract", "")

In [32]:
def is_ml_related(title, abstract):
  prompt = f"""
  Given the abstract of the wikipedia article,
  determine if the articile is machine-learning or language model related

  [Rules]
  - Exclue articles about specific individuals
  - Exclue articles that are general knowledge and not related to machine-learning or language model
  - Exclue articles about the developer, company
  - Answer True if the articile is lreatred to machine-learning or langugae model, else False

  [title]
  {title}

  [abstract]
  {abstract}

  [answer]
  """

  return (get_response(prompt).lower().find("true") > -1)

In [34]:
idx = 0
links_lm[0]

'1.58-bit large language model'

In [35]:
title = links_lm[0]
abstract = get_article_abstract(title)

In [36]:
abstract

'A 1.58-bit Large Language Model (1.58-bit LLM, also ternary LLM) is a version of a transformer large language model with weights using only three values: -1, 0, and +1. This restriction theoretically allows the model to replace costly multiplications with additions and reduce the storage memory. Since the end-task performance and perplexity of the 1.58-bit LLMs, at least for smaller model sizes (up to 3-4B parameters), are close to their "full precision" (16-bit FP16 or BF16) counterparts, this design allows reaching the same artificial intelligence goals with much lower hardware requirements, latency, and training effort.\nThe name comes from a fact that a single trit, a ternary arithmetic equivalent of a bit that can take the {-1, 0, 1} values, carries \n  \n    \n      \n        l\n        o\n        \n          g\n          \n            2\n          \n        \n        3\n        ≈\n        1.58\n      \n    \n    {\\displaystyle log_{2}3\\approx 1.58}\n  \n bits of information. 

In [37]:
is_ml_related(title, abstract)

True

In [38]:
links_lm

['1.58-bit large language model',
 '1.58-bit large language model',
 '15.ai',
 'AI-complete',
 'AI agent',
 'AI explainability',
 'API',
 'Action selection',
 'Activation function',
 'Active learning (machine learning)',
 'Adobe Firefly',
 'Adversarial machine learning',
 'Aidan Gomez',
 'Alan Turing',
 'AlexNet',
 'Alex Graves (computer scientist)',
 'Alex Krizhevsky',
 'Algorithmic bias',
 'Allen Newell',
 'AlphaFold',
 'AlphaGo',
 'AlphaZero',
 'Andrej Karpathy',
 'Andrew Ng',
 'Anomaly detection',
 'Apache License',
 'Apprenticeship learning',
 'ArXiv (identifier)',
 'Argument mining',
 'Ari Holtzman',
 'Array (data structure)',
 'Artificial consciousness',
 'Artificial general intelligence',
 'Artificial human companion',
 'Artificial intelligence',
 'Artificial intelligence and copyright',
 'Artificial neural network',
 'Ashish Vaswani',
 'Association for Computing Machinery',
 'Association rule learning',
 'Attention (machine learning)',
 'Attention Is All You Need',
 'Aurora (t

In [42]:
title = links_lm[2]
abstract = get_article_abstract(title)
is_ml_related(title, abstract)

False